<a href="https://colab.research.google.com/github/rshaikh95/Diabetes_Dataset/blob/main/Diabetic_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Commands to install some of the libraries in-case if they are not installed
# Any other library that needs to be installed just use: !pip install <library name>
!pip install seaborn
!pip install missingno
!pip install xgboost
!pip install catboost
!pip install regex
!pip install sklearn
!pip install pandas
!pip install numpy
!pip install imblearn
!pip install lightgbm

### Import Libraries

In [23]:
import pandas as pd   # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np   # linear algebra
import matplotlib.pyplot as plt  #graphs and plots
import seaborn as sns   #data visualizations
import csv # Some extra functionalities for csv  files - reading it as a dictionary
from lightgbm import LGBMClassifier #sklearn is for machine learning and statistical modeling including classification, regression, clustering and dimensionality reduction

from sklearn.model_selection import train_test_split, cross_validate   #break up dataset into train and test sets

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler


# importing python library for working with missing data
import missingno as msno
# To install missingno use: !pip install missingno
import re    # This library is used to perform regex pattern matching

# import various functions from sklearn
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
import xgboost as xgb
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, classification_report, make_scorer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split

In [25]:
from sklearn.model_selection import KFold,cross_val_score, RepeatedStratifiedKFold,StratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import OneHotEncoder,StandardScaler,PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer,SimpleImputer
from sklearn.compose import make_column_transformer
from imblearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer
from sklearn.dummy import DummyClassifier
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier


import plotly
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as py
from plotly.offline import iplot
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

import warnings
warnings.filterwarnings("ignore")

### Read CSV File

In [11]:
diabetes = pd.read_csv('diabetic_data.csv')

In [ ]:
diabetes.head(n=50)

### Y Values: Readmitted

In [ ]:
y = diabetes['readmitted']
print(f'Percentage of patient readmitted for diabetes (>30 days): % {round(y.value_counts(normalize=True)[1]*100,2)} --> ({y.value_counts()[1]} patient)\nPercentage of patient not readmitted for diabetes: % {round(y.value_counts(normalize=True)[0]*100,2)} --> ({y.value_counts()[0]} patient)\nPercentage of patient readmitted for diabetes (<30 days): % {round(y.value_counts(normalize=True)[2]*100,2)} --> ({y.value_counts()[2]} patient)')

### Figure of Diabetes Readmitted ( NO , <30 , >30)

In [26]:
fig = px.histogram(diabetes, x="readmitted", title='Diabetes', width=400, height=400)
fig.show()

In [ ]:
 diabetes.info()

## Defined missing data


In [ ]:
diabetes.duplicated().sum()

Found "?" to be missing value: replaced with NA

In [ ]:
diabetes.replace("?", np.nan, inplace=True)
diabetes

In [ ]:
def missing (diabetes):
    missing_number = diabetes.isnull().sum().sort_values(ascending=False)
    missing_percent = (diabetes.isnull().sum()/diabetes.isnull().count()).sort_values(ascending=False)
    missing_values = pd.concat([missing_number, missing_percent], axis=1, keys=['Missing_Number', 'Missing_Percent'])
    return missing_values


missing(diabetes)

In [19]:

diabetes.drop(['weight'], axis = 1, inplace = True)
# dropping weight due to 96% missing

In [ ]:
def missing (diabetes):
    missing_number = diabetes.isnull().sum().sort_values(ascending=False)
    missing_percent = (diabetes.isnull().sum()/diabetes.isnull().count()).sort_values(ascending=False)
    missing_values = pd.concat([missing_number, missing_percent], axis=1, keys=['Missing_Number', 'Missing_Percent'])
    return missing_values


missing(diabetes)

In [ ]:
diabetes.dropna(inplace=True)
diabetes
# dropped all NA values

### Categorical and Numerical variables

In [27]:
categorical = ['race', 'gender', 'age', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
               'payer_code', 'medical_specialty', 'diag_1', 'diag_2', 'diag_3',
                'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride',
               'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone',
              'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide',
              'examide', 'citoglipton', 'insulin', 'glyburide-metformin', 'glipizide-metformin',
               'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone',
               'change', 'diabetesMed', 'readmitted']

numerical = ['time_in_hospital','num_lab_procedures', 'num_procedures','num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses']

In [ ]:
diabetes[numerical].describe()

## Skewness

In [ ]:
diabetes[numerical].skew()

Based on skewness, num of lab procedures and number of diagnoses has a slight left skew (negative) while the rest have right skew (positive)

## Univariate Analysis

In [ ]:
diabetes[numerical].hist(figsize=(20,10));

Categorical Variables:

               'payer_code', 'medical_specialty', 'diag_1', 'diag_2', 'diag_3',
                'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride',
               'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone',
              'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide',
              'examide', 'citoglipton', 'insulin', 'glyburide-metformin', 'glipizide-metformin',
               'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone',
               'change', 'diabetesMed', 'readmitted

In [ ]:
categorical = ['race', 'gender', 'age', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
               'payer_code', 'medical_specialty', 'diag_1', 'diag_2', 'diag_3',
                'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride',
               'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone',
              'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide',
              'examide', 'citoglipton', 'insulin', 'glyburide-metformin', 'glipizide-metformin',
               'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone',
               'change', 'diabetesMed', 'readmitted']
# loop for each column instead of repeating code
for column in categorical:
 value_counts = diabetes[column].value_counts(normalize=True)
 # Printing the value counts
 print(f'{column}\n{value_counts}\n')
 # Creating a histogram using Plotly Express
 fig = px.histogram(diabetes, x=column, title=column, width=500, height=500)
 fig.show()

In [ ]:
print (f'{round(diabetes["gender"].value_counts(normalize=True)*100,2)}')
fig = px.histogram(diabetes, x="gender", title='Gender', width=500, height=500)
fig.show()

In [ ]:
print (f'{round(diabetes["race"].value_counts(normalize=True)*100,2)}')
fig = px.histogram(diabetes, x="race", title='Race', width=500, height=500)
fig.show()

In [ ]:
print (f'{round(diabetes["age"].value_counts(normalize=True)*100,2)}')
fig = px.histogram(diabetes, x="age", title='Age', width=500, height=500)
fig.show()

In [ ]:
print (f'{round(diabetes["max_glu_serum"].value_counts(normalize=True)*100,2)}')
fig = px.histogram(diabetes, x="max_glu_serum", title='Max Glucose Serum', width=500, height=500)
fig.show()

In [ ]:
print (f'{round(diabetes["A1Cresult"].value_counts(normalize=True)*100,2)}')
fig = px.histogram(diabetes, x="A1Cresult", title='A1C Results', width=500, height=500)
fig.show()

In [ ]:
print (f'{round(diabetes["metformin"].value_counts(normalize=True)*100,2)}')
fig = px.histogram(diabetes, x="metformin", title='Metformin', width=500, height=500)
fig.show()

In [ ]:
print (f'{round(diabetes["readmitted"].value_counts(normalize=True)*100,2)}')
fig = px.histogram(diabetes, x="readmitted", title='Readmitted', width=500, height=500)
fig.show()

## Bivariate Analysis

In [ ]:
diabetes["readmitted"].unique()

In [40]:
# Replacing values
diabetes["readmitted"] = diabetes["readmitted"].replace({"NO": 0, "<30": 1, ">30": 2})

In [ ]:
diabetes["readmitted"].value_counts()

## Race and Readmission

In [ ]:
print (f'A person with Caucasian race has a probability of {round(diabetes[diabetes["race"]=="Caucasian"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person with African American race has a probability of {round(diabetes[diabetes["race"]=="AfricanAmerican"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person with Hispanic race has a probability of {round(diabetes[diabetes["race"]=="Hispanic"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person with Asian race has a probability of {round(diabetes[diabetes["race"]=="Asian"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person with Other race has a probability of {round(diabetes[diabetes["race"]=="Other"]["readmitted"].mean()*100,2)} % to be readmitted')

In [ ]:
fig = px.histogram(diabetes, x="race", color="readmitted",width=600, height=600)
fig.show()

## Gender and Readmission

In [ ]:
print (f'A person with Male gender has a probability of {round(diabetes[diabetes["gender"]=="Male"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person with Female gender has a probability of {round(diabetes[diabetes["gender"]=="Female"]["readmitted"].mean()*100,2)} % to be readmitted')

In [ ]:
fig = px.histogram(diabetes, x="gender", color="readmitted",width=600, height=600)
fig.show()


## Age and Readmission

In [ ]:
print (f'A person [70-80) has a probability of {round(diabetes[diabetes["age"]=="[70-80)"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person [60-70) has a probability of {round(diabetes[diabetes["age"]=="[60-70)"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person [80-90) has a probability of {round(diabetes[diabetes["age"]=="[80-90)"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person [50-60) has a probability of {round(diabetes[diabetes["age"]=="[50-60)"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person [40-50) has a probability of {round(diabetes[diabetes["age"]=="[40-50)"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person [30-40) has a probability of {round(diabetes[diabetes["age"]=="[30-40)"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person [90-100) has a probability of {round(diabetes[diabetes["age"]=="[90-100)"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person [20-30) has a probability of {round(diabetes[diabetes["age"]=="[20-30)"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person [10-20) has a probability of {round(diabetes[diabetes["age"]=="[10-20)"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person [0-10) has a probability of {round(diabetes[diabetes["age"]=="[0-10)"]["readmitted"].mean()*100,2)} % to be readmitted')

In [ ]:
fig = px.histogram(diabetes, x="age", color="readmitted",width=600, height=600)
fig.show()


## Metformin and Readmission

In [ ]:
print (f'A person with No metformin has a probability of {round(diabetes[diabetes["metformin"]=="No"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person with Steady metformin has a probability of {round(diabetes[diabetes["metformin"]=="Steady"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person with Up metformin has a probability of {round(diabetes[diabetes["metformin"]=="Up"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person with Down metformin has a probability of {round(diabetes[diabetes["metformin"]=="Down"]["readmitted"].mean()*100,2)} % to be readmitted')


In [ ]:
fig = px.histogram(diabetes, x="metformin", color="readmitted",width=600, height=600)
fig.show()

## A1C Results and Readmission

In [ ]:
print (f'A person with None A1C result has a probability of {round(diabetes[diabetes["A1Cresult"]=="None"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person with >8 A1C result has a probability of {round(diabetes[diabetes["A1Cresult"]==">8"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person with Norm A1C result has a probability of {round(diabetes[diabetes["A1Cresult"]=="Norm"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person with >7 A1C result has a probability of {round(diabetes[diabetes["A1Cresult"]==">7"]["readmitted"].mean()*100,2)} % to be readmitted')


In [ ]:
fig = px.histogram(diabetes, x="A1Cresult", color="readmitted",width=600, height=600)
fig.show()

## Metrics of Importance

In [ ]:
from sklearn.metrics import mutual_info_score
def cat_mut_inf(series):
    return mutual_info_score(series, diabetes['readmitted'])

diabetes_cat = diabetes[categorical].apply(cat_mut_inf)
diabetes_cat = diabetes_cat.sort_values(ascending=False).to_frame(name='mutual_info_score')
diabetes_cat

## Correlation Matrix & Scatter Plots


In [ ]:
diabetes[numerical].corr()

In [ ]:
diabetes.groupby('readmitted')[numerical].mean()

In [ ]:
numerical = ['time_in_hospital','num_lab_procedures', 'num_procedures','num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses']
diabetes[numerical].corr()

In [ ]:
 fig = px.scatter(diabetes, x='time_in_hospital', y='num_lab_procedures', title='Time in Hospital vs Num Lab Procedures')
fig.show()

In [ ]:
fig = px.scatter(diabetes, x='time_in_hospital', y='num_medications', title='Time in Hospitals vs Num of Medication')
fig.show()

In [ ]:

fig = px.scatter(diabetes, x='time_in_hospital', y='number_outpatient', title='Time in Hospital vs Num Outpatient')
fig.show()

In [ ]:
fig = px.scatter(diabetes, x='time_in_hospital', y='number_emergency', title='Time in Hospital vs Num Emergency')
fig.show()

In [ ]:
fig = px.scatter(diabetes, x='time_in_hospital', y='number_inpatient', title='Time in Hospital vs Num Inpatient')
fig.show()

In [ ]:
fig = px.scatter(diabetes, x='time_in_hospital', y='number_diagnoses', title='Time in Hospital vs Num Diagnoses')
fig.show()

## Breaking the data up into Train & Test

In [63]:
# Using get_dummies from the pandas library to convert categorical feature to numeric
diabetes = pd.get_dummies(diabetes, drop_first = False)

In [64]:
train_df, valid_df, test_df = np.split(diabetes.sample(frac=1, random_state=42),
                                       [int(.7*len(diabetes)), int(0.85*len(diabetes))])
train_df = train_df.reset_index(drop = True)
valid_df = valid_df.reset_index(drop = True)
test_df = test_df.reset_index(drop = True)

In [67]:
diabetes.readmitted.value_counts()


0    15154
2     8744
1     2857
Name: readmitted, dtype: int64

In [69]:
train_df.readmitted.value_counts()

0    10535
2     6177
1     2016
Name: readmitted, dtype: int64

In [70]:
valid_df.readmitted.value_counts()

0    2304
2    1280
1     429
Name: readmitted, dtype: int64

In [71]:
test_df.readmitted.value_counts()

0    2315
2    1287
1     412
Name: readmitted, dtype: int64

In [72]:
def calc_prevalence(y_actual):

    '''
    This function is to understand the ratio/distribution of the classes that we are going to predict for.

    Params:
    1. y_actual: The target feature

    Return:
    1. (sum(y_actual)/len(y_actual)): The ratio of the postive class in the comlpete data.
    '''

    return (sum(y_actual)/len(y_actual))

In [74]:
# split the training data into positive and negative based on 'readmitted'
rows_pos = diabetes.readmitted == 1
df_train_pos = diabetes.loc[rows_pos]
df_train_neg = diabetes.loc[~rows_pos]
# merge the balanced data with replacement
readmitted_df_balanced = pd.concat([df_train_pos, df_train_neg.sample(n=len(df_train_pos), replace=True, random_state=111)], axis=0)
# shuffle the order of training samples
readmitted_df_balanced = readmitted_df_balanced.sample(n=len(readmitted_df_balanced), random_state=42).reset_index(drop=True)
print('Train balanced prevalence(n = %d): %.3f' % (len(readmitted_df_balanced), \
 calc_prevalence(readmitted_df_balanced.readmitted.values)))

Train balanced prevalence(n = 5714): 0.874


In [75]:
readmitted_df_balanced.readmitted.value_counts()


1    2857
0    1789
2    1068
Name: readmitted, dtype: int64

In [76]:
X_train = readmitted_df_balanced.drop('readmitted',axis=1)
y_train = readmitted_df_balanced['readmitted']
X_valid = valid_df.drop('readmitted',axis=1)
y_valid = valid_df['readmitted']
X_test = test_df.drop('readmitted',axis=1)
y_test = test_df['readmitted']


In [79]:
scaler=StandardScaler()
X_train[['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency','number_inpatient', 'number_diagnoses']] = pd.DataFrame(scaler.fit_transform(X_train[['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency','number_inpatient', 'number_diagnoses']]),columns=['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency','number_inpatient', 'number_diagnoses'])
X_valid[['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses']] = pd.DataFrame(scaler.fit_transform(X_valid[['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency','number_inpatient', 'number_diagnoses']]),columns=['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency','number_inpatient', 'number_diagnoses'])
X_test[['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses']] = pd.DataFrame(scaler.fit_transform(X_test[['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency','number_inpatient', 'number_diagnoses']]),columns=['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency','number_inpatient', 'number_diagnoses'])

## Creating and Understanding Models

In [81]:
def calc_specificity(y_actual, y_pred, thresh):
 # calculates specificity
 return sum((y_pred < thresh) & (y_actual == 0)) /sum(y_actual ==0)
def print_report(y_actual, y_pred, thresh = 0.5):

 '''
 This function calculates all the metrics to asses the machine learning models.

 Params:
 1. y_actual: The actual values for the target variable.
 2. y_pred: The predicted values for the target variable.
 3. thresh: The threshold for the probability to be considered as a positive class. Default value 0.5

 Return:
 1. AUC
 2. Accuracy
 3. Recall
 4. Precision
 5. Specificity
 '''

 auc = roc_auc_score(y_actual, y_pred)
 accuracy = accuracy_score(y_actual, (y_pred > thresh))
 recall = recall_score(y_actual, (y_pred > thresh))
 precision = precision_score(y_actual, (y_pred > thresh))
 specificity = calc_specificity(y_actual, y_pred, thresh)
 print('AUC:%.3f'%auc)
 print('accuracy:%.3f'%accuracy)
 print('recall:%.3f'%recall)
 print('precision:%.3f'%precision)
 print('specificity:%.3f'%specificity)
 print('prevalence:%.3f'%calc_prevalence(y_actual))
 print(' ')
 return auc, accuracy, recall, precision, specificity

## Linear Regression

In [82]:
lnr = LinearRegression()
lnr.fit(X_train, y_train)
y_valid_preds = lnr.predict(X_valid)


In [83]:
y_valid_preds

array([0.8083177 , 1.10111918, 0.65382558, ..., 0.83423209, 0.62054869,
       1.7850372 ])

## Logistic Regression

In [85]:
lr=LogisticRegression(random_state = 42, solver = 'newton-cg', max_iter = 200)
lr.fit(X_train, y_train)

y_valid_preds = lr.predict_proba(X_valid)[:,1]

print('Metrics for Validation data:')

lr_valid_auc, lr_valid_accuracy, lr_valid_recall, \
    lr_valid_precision, lr_valid_specificity = print_report(y_valid,y_valid_preds, 0.5)

Metrics for Validation data:


TypeError: print_report() got an unexpected keyword argument 'average'

In [86]:
dc_clf = DecisionTreeClassifier(random_state=42, max_depth = 10)
dc_clf.fit(X_train, y_train)

dc_preds_proba = dc_clf.predict_proba(X_valid)[:,1]
dc_preds = dc_clf.predict(X_valid)

lr_valid_auc, lr_valid_accuracy, lr_valid_recall, \
    lr_valid_precision, lr_valid_specificity = print_report(y_valid,dc_preds_proba, 0.5)

ValueError: multi_class must be in ('ovo', 'ovr')